In [1]:
import datajoint as dj
import numpy as np
import time

from meshparty import trimesh_io

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.FilteredSkeleton()
#ta3p100.NeuriteRawSkeletonReach()

In [4]:
# #how to extract the bounding box

# key = dict(segmentation=2,segment_id=648518346341351731)
# skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
# print(skeleton_edges.shape)
# vertices = skeleton_edges.reshape(-1,3)
# vertices
# mins = np.amin(vertices, axis=0)
# maxs = np.amax(vertices, axis=0)
# print("mins,maxs = " + str((mins,maxs)))
# diff = maxs-mins
# volume = diff[0]*diff[1]*diff[2]
# print("volume = " + str(volume/1000000000))

In [5]:
# #how to extract the lengths
# filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  skeleton_edges]
# sum(filtered_edges_distance_lin_alg)

In [6]:
@schema
class NeuriteRawSkeletonReach(dj.Computed):
    definition="""
    -> ta3p100.FilteredSkeleton
    ---
    bbox_volume   :double unsigned #length of the entire skeleton, raw_volume/10^9
    bbox_corners     :blob #bounding box corners for the soma mesh (min,max)
    skeleton_length    :double unsigned #length of the entire skeleton
    """
    
    def make(self, key):
        start_time = time.time()
        print(str(key["segment_id"]) + ":")
        #get the skeleton edges
        skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
        vertices = skeleton_edges.reshape(-1,3)
        
        mins = np.amin(vertices, axis=0)
        maxs = np.amax(vertices, axis=0)
        #print("mins,maxs = " + str((mins,maxs)))
        key["bbox_corners"] = (mins,maxs)
        
        diff = maxs-mins
        volume = diff[0]*diff[1]*diff[2]
        #print("volume = " + str(volume/1000000000))
        key["bbox_volume"] = volume/1000000000
        
        filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  skeleton_edges]
        total_length = sum(filtered_edges_distance_lin_alg)
        
        key["skeleton_length"] = total_length
        
        self.insert1(key,skip_duplicates=True)
        
        print(f"Total Time: {time.time()-start_time}")
        print()
        
    
    

In [7]:
#(schema.jobs & "table_name='__neurite_raw_skeleton_reach'").delete()

In [8]:
start_time = time.time()
NeuriteRawSkeletonReach.populate(reserve_jobs=True)
print(f"Total time for populate: {time.time()-start_time}")

648518346341362205:
Total Time: 0.04535055160522461

648518346341362268:
Total Time: 0.05237150192260742

648518346341362281:
Total Time: 0.03442215919494629

648518346341362283:
Total Time: 0.03728485107421875

648518346341362301:
Total Time: 0.018629074096679688

648518346341362302:
Total Time: 0.038774728775024414

648518346341362451:
Total Time: 0.043898582458496094

648518346341362453:
Total Time: 0.04080057144165039

648518346341362455:
Total Time: 0.04342985153198242

648518346341362461:
Total Time: 0.014669656753540039

648518346341362466:
Total Time: 0.04554605484008789

648518346341362471:
Total Time: 0.028052806854248047

648518346341362477:
Total Time: 0.023719310760498047

648518346341362499:
Total Time: 0.03519320487976074

648518346341362517:
Total Time: 0.04882526397705078

648518346341362521:
Total Time: 0.05053091049194336

648518346341362532:
Total Time: 0.036721229553222656

648518346341362537:
Total Time: 0.03220343589782715

648518346341362542:
Total Time: 0.04891